##Cross validation & GridSearch

- 테스트 세트를 사용해 계속 성능을 확인하다 보면 점점 테스트 세트에 맞추게 됨.
- 테스트 세트로 일반화 성능을 올바르게 예측하려면 가능한 한 테스트 세트를 사용하지 말아야 한다.

##Validation set

- 테스트 세트를 사용하지 않고 이를 측정하는 방법은 훈련 세트를 다시 나누는 것

In [1]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

class column 을 target으로 사용하고 나머지 column feature array에 저장

In [3]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [8]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [10]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [11]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [12]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


##croo validation

교차 검증은 검증 세트를 떼어 내어 평가하는 과정을 여러 번 반복한다.

## k-fold cross validation
- 훈련 세트를 k 부분으로 나눠서 교차 검증을 수행하는 것

In [14]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00991344, 0.00707388, 0.00790262, 0.00716352, 0.0070498 ]), 'score_time': array([0.00092363, 0.00133538, 0.00073481, 0.0007298 , 0.00072789]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


교차 검증의 최종 점수는 test_score 키에 담긴 5개의 점수를 평균하여 얻을 수 있다.

이름은 test_score지만 검증 폴드의 점수이다.

In [15]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


##splitter

- 사이킷런의 분할기는 교차 검증에서 폴드를 어떻게 나눌지 결정해 준다.
- cross_validate() 함수는 기본적으로 회귀 모델일 경우 KFold 부할기를 사용하고

 분류 모델일 경우 타깃 클래스를 골고루 나누기 위해 StratifiedKFold를 사용한다.


In [16]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [17]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


##Hyperparameter tuning
- 모델이 학습할 수 없어서 사용자가 지정해야지만 하는 파라미터를 하이퍼파라미터라고 한다.

##Grid Search

##GridSearchCV

- 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행한다.

- n_jobs =-1 : CPU 코어수를 지정하는데 -1은 모든 코어를 사용한다.

In [18]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [19]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [20]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [21]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [22]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [24]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


##정리

1. 먼저 탐색할 매개변수를 지정한다.

2. 훈련 세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾는다. 이 조합은 그리드 서치 객체에 저장된다.

3. 그리드 서치는 최상의 매개변수에서 (교차 검증에 사용한 훈련 세트가 아니라) 전체 훈련 세트를 사용해 최종 모델을 훈련한다. 이 모델도 서치 객체에 저장된다.

##조금 더 복잡한 매개변수 조합

##min_impurity_decrease

- 노드를 분할하기 위한 불순도 감소 최소량을 지정한다.

In [25]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split' : range(2, 100, 10)
          }

In [26]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [27]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [28]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


##Random Search

- 매개변수의 값이 수치일 때 값의 범위나 간격을 미리 정하기 어려울 때
- 혹은 너무 많은 매개변수 조건이 있어 그리드 서치 수행 시간이 오래 걸릴 때

##scipy
-싸이파이는 파이썬의 핵심 과학 라이브러리 중 하나. 

In [30]:
## uniform - 실수, randint - 정수
from scipy.stats import uniform, randint

In [31]:
rgen = randint(0, 10)
rgen.rvs(10)

array([5, 2, 6, 2, 2, 2, 6, 0, 7, 1])

In [32]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 94, 103,  96,  98, 102,  87, 109,  96, 111, 104]))

In [33]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.23771524, 0.11009619, 0.41637507, 0.84998318, 0.37402672,
       0.748545  , 0.31241769, 0.80616776, 0.32643873, 0.97535919])

In [34]:
params = {'min_impurity_decrease' : uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [36]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f297ce74c10>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f297ce74160>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f297ce75f40>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f297ce75310>},
                   random_state=42)

In [37]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [38]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [39]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86


##최적의 모델을 위한 하이퍼파라미터 탐색

- 레드 와인과 화이트 와인을 선별하는 작업의 성능을 끌어올리기 위해 결정 트리의 다양한 하이퍼파라미터를 시도해 봐야 한다. 이런 과정에서 테스트 세트를 사용하면 결국 테스트 세트에 맞춰 훈려하는 효과를 만든다.

- 테스트 세트는 최종 모델을 선택할 때까지 사용하지 말아야 한다. 테스트 세트를 사용하지 않고 모델을 평가하려면 또 다른 세트가 필요하다. 이를 검증 세트라고 부른다. 혹은 개발 (dev set)라고도 부른다. 검증 세트는 훈련 세트 중 일부를 다시 덜어 내어 만든다.

- 검증 세트가 크지 않다면 어떻게 데이터를 나누었는지에 따라 검증 점수가 둘쭉날쭉할 것이다. 훈련한 모델의 성능을 안정적으로 평가하기 위해 검증 세트를 한 번 나누어 모델을 평가하는 것에 그치지 않고 여러 번 반복할 수 있다. 이를 교차 검증이라고 한다.

- 보통 훈련 세트를 5등분 혹은 10등분 한다. 나누어진 한 덩어리를 폴드라고 부르며 한 폴드씩 돌아가면서 검증 세트의 역할을 한다. 따라서 전체적으로 5개 혹은 10개의 모델을 만든다. 최정 검증 점수는 모든 폴드의 검증 점수를 평군하여 계산한다.

- 교차 검증을 사용해 다양한 하이퍼파라미터를 탐색한다. 머신러닝 라이브러리에서는 클래스와 메서드의 매개변수를 바꾸어 모델을 훈련하고 평가해 보는 작업이다. 이런 과정은 때론 지루하고 반복적이다. 테스트하고 싶은 리스트를 만들어 이 과정을 자동화하는 그리드 서치를 사용하면 편리하다.

- 매개변수 값이 수치형이고 특히 연속적인 실숫값이라며 싸이파이의 확률 분포 객체를 전달하여 특정 범위 내에서 지정된 횟수만큼 매개변수 후보 값을 샘플링하여 교차 검증을 시도할 수 있다. 이는 한정된 자원을 최대한 활용하여 효율적으로 하이퍼파라미터 공간을 탐색할 수 있는 아주 좋은 도구이다.

